In [1]:
import csv
import os
from finalproject.class_dataset import ChestDataset
import pandas as pd
import glob
import math
import pickle
from PIL import ImageFile
import time
from tqdm import tqdm
from IPython.core.display import Image, display

from sklearn.datasets import load_files
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras import optimizers
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.applications import DenseNet121
from keras import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard


Using TensorFlow backend.


In [2]:
csvfile = 'finalproject/data/Data_Entry_2017.csv'
train_list = open('finalproject/data/train_val_list.txt','r').read().split('\n')
test_list = open('finalproject/data/test_list.txt','r').read().split('\n')

In [3]:
df = pd.read_csv(csvfile)

In [4]:
df_uni = df[~df['Finding Labels'].str.contains('\|')]

In [5]:
data_dir = 'image/images/'
dataset = ChestDataset(data_dir,df_uni,4)
dataset.reset_folder()

/home/yair/Documents/ITC/Final/finalproject/class_dataset.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.reader['exists'] = self.exists


In [6]:
train_dt,test_dt = dataset.train_test(train_list,test_list)
train_dt.create_tree()
test_dt.create_tree()

/home/yair/Documents/ITC/Final/finalproject/class_dataset.py:139: UserWarning: file not found
  warnings.warn('file not found')
/home/yair/Documents/ITC/Final/finalproject/class_dataset.py:114: UserWarning: Image not found in folder
  warnings.warn('Image not found in folder')


In [7]:
# Add GPU options
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))
NAME = 'Binary_chest_cnn_64x3_{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
#tensorboard --logdir=logs/ --host localhost --port 8088

In [8]:
def load_dataset(path):
    """Returns the path and the Label from the folder"""
    data = load_files(path)
    chest_files = np.array(data['filenames'])
    chest_targets = np_utils.to_categorical(np.array(data['target']), 2)
    return chest_files, chest_targets

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [9]:
# train_files,train_targets = load_dataset(train_dt.dir)
# test_files,test_targets = load_dataset(test_dt.dir)

# labels = dataset.labels

# ImageFile.LOAD_TRUNCATED_IMAGES = True
# train_tensors = paths_to_tensor(train_files).astype('float32')/255
# test_tensors = paths_to_tensor(test_files).astype('float32')/255
# print(train_tensors.shape, train_targets.shape)

In [10]:
train_files = train_dt.image_path
test_files = test_dt.image_path
train_folder = train_dt.dir
test_folder = test_dt.dir

In [11]:
print('Statistics about the Dataset:\n')
print('There are %d total chest deseases.' % len(set(dataset.labels)))
print('There are %s total chest images.\n' % np.sum(dataset.exists))
print('There are %d training chest images.' % np.sum(train_dt.exists))
# print('There are %d validation dog images.' % len(valid_files))
print('There are %d test chest images.'% np.sum(test_dt.exists))

Statistics about the Dataset:

There are 15 total chest deseases.
There are 0 total chest images.

There are 10786 training chest images.
There are 1887 test chest images.


In [12]:
img_width,img_height = 365,365
densenet = DenseNet121(weights='imagenet', include_top=False,input_shape = (img_width, img_height, 3))

# Freeze some layers
for layer in densenet.layers[:]:
    layer.trainable = False
    
# Create the model
model = models.Sequential()

# Add the vgg convolutional base model

model.add(densenet)

# Add new layers
model.add(Flatten())
model.add(Dense(248))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.248))
model.add(Dense(15, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 11, 11, 1024)      7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 123904)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 248)               30728440  
_________________________________________________________________
batch_normalization_1 (Batch (None, 248)               992       
_________________________________________________________________
activation_1 (Activation)    (None, 248)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 248)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                3735      
Total para

In [16]:
train_datagen = ImageDataGenerator()

validation_datagen = ImageDataGenerator()

# Change the batchsize according to your system RAM
train_batchsize = 10
val_batchsize = 10

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(img_height, img_width),
    batch_size=train_batchsize,
    class_mode='categorical')

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

validation_generator = validation_datagen.flow_from_directory(
    test_folder,
    target_size=(img_height, img_width),
    batch_size=val_batchsize,
    class_mode='categorical',
    shuffle=False)

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    verbose=1,
    use_multiprocessing=True)


Found 10786 images belonging to 15 classes.
Found 1887 images belonging to 15 classes.
Epoch 1/2
  86/1078 [=>............................] - ETA: 4:45:16 - loss: 0.1608 - acc: 0.9529

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yair/anaconda3/envs/Final/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yair/anaconda3/envs/Final/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yair/anaconda3/envs/Final/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yair/anaconda3/envs/Final/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/yair/anaconda3/envs/Final/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yair/anaconda3/envs/Final/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/yair/anaconda3/envs/Final/lib/python3.6/multiprocessing/pool.py", line 108, in

Epoch 1/2
Epoch 1/2


KeyboardInterrupt: 